This notebook pulls 2010 census data and attempts to generate a large set of points that approximates a smooth surface

In [14]:
# Declare static variables

n=50 # The number of points to assign to each census block
BINOMIAL_TRIALS = 25 # The number of trials in the binomial distribution used for weighting points in blocks. The higher the value, the more evenly distributed the population points will be through the census block
BINOMIAL_SUCCESS = 0.3 # The probability of success for each trial in he weight assignment. Must be <=1


In [2]:
# Import libraries

import pandas as pd
import geopandas
import numpy as np
import requests
from io import BytesIO
import folium
from IPython.display import clear_output
from itertools import chain

In [3]:
# Request shapefile data for 2010 census tracts and convert to geopandas dataframe

# Shapefile url
data_url = 'https://www2.census.gov/geo/tiger/GENZ2010/gz_2010_10_140_00_500k.zip'


# Request data
data = requests.get(data_url)
# convert to pandas dataframe
tract_data = geopandas.read_file(BytesIO(data.content))

In [4]:
# Request shapefile data for 2010 census tracts and convert to geopandas dataframe

# Shapefile url
data_url = 'https://www2.census.gov/geo/tiger/TIGER2010/TABBLOCK/2010/tl_2010_10_tabblock10.zip'


# Request data
data = requests.get(data_url)
# convert to pandas dataframe
block_data = geopandas.read_file(BytesIO(data.content))

In [5]:
# For each census block, create a bounding box
block_bounds = block_data["geometry"].bounds

# Attch GEOID to boundaries
block_bounds = block_data[["GEOID10","geometry"]].merge(block_bounds, left_index=True, right_index=True)

In [6]:
# Fit a 2D Gaussian distribution over the bounding boxes

# Takes in a row of 'block_bounds' and outputs a 2D Gaussian distribution of 'n' points over the bounding box, as well as the GEOID
def get_points(row,n):
    print(f"Processing Block {row['GEOID10']}...")
    # 'i' is the total number of points left to assign
    i=n
    # 'points_return' is the list of all points for the block
    # TODO: CRS is hardcoded
    points_return = geopandas.GeoSeries(crs="EPSG:4269")
    # Allocate points until n have been assigned
    while i > 0:
        # Generates a uniform distribution for the y-axis located at the center of the box
        pointsy = np.random.uniform(low=row["miny"], high=row["maxy"], size=i)
        # Generates a uniform distribution for the x-axis located at the center of the box
        pointsx = np.random.uniform(low=row["minx"], high=row["maxx"], size=i)
        # Convert the points to Shapely points
        points = geopandas.GeoSeries(geopandas.points_from_xy(pointsx, pointsy, crs="EPSG:4269"))
        # Check if the points are inside the block
        point_checks = points.within(row["geometry"])
        # Add found points to our list
        points_return = geopandas.GeoSeries(pd.concat([points_return, points[point_checks]], ignore_index=True), crs=points_return.crs)
        # Set 'i' equal to the number of missed points
        i = n - points_return.size
    
    # Generates a binomial distribution of weights
    weights = np.random.binomial(n=BINOMIAL_TRIALS, p=BINOMIAL_SUCCESS, size=n)
    # Normalize weights so that they sum to 1
    weights = weights / np.sum(weights)
    # Sort the weights based on distance from the mean
    weights = weights[np.argsort(np.abs(weights - np.mean(weights)))]
    # Generate a series containing the distance from each point to the centroid
    distances = points_return.distance(row["geometry"].centroid)
    # Create a column for the index of the point and sort by distance
    distances = distances.reset_index(name="distance").sort_values(by="distance")
    # Assign a weight to each point
    distances["weight"] = weights
    # Merge weights onto points
    points_return = pd.merge(left=points_return.rename("geometry"), right=distances, how="left", right_on="index", left_index=True)[["geometry", "weight"]]
    # Clear warnings from notebook output to prevent crash
    clear_output()
    # Return an array with every point in the cloud, the weights for each point and the GEOID
    return list(chain(points_return["geometry"].values, points_return["weight"].values, [row["GEOID10"]]))
    

In [15]:
# Fit a Gaussian distribution to each block
point_cloud = block_bounds.apply(get_points, axis=1, args=(n,), result_type='expand')

# Rename columns of the pointcloud
point_cloud.columns = ['point_' + str(x) if x<n else 'weight_' + str(x-n) if x<2*n else 'GEOID' for x in point_cloud.columns]

point_cloud



# TODO: Use distribution weights in below computations

,point_0,point_1,point_2,point_3,point_4,point_5,point_6,point_7,point_8,point_9,...,weight_41,weight_42,weight_43,weight_44,weight_45,weight_46,weight_47,weight_48,weight_49,GEOID
0,POINT (-75.48551389930064 39.122244015158834),POINT (-75.48628082108375 39.120542980309345),POINT (-75.48669388205316 39.1217852887391),POINT (-75.4861606355758 39.121906171200585),POINT (-75.48521328375543 39.12251929307355),POINT (-75.48590034460919 39.12208336375866),POINT (-75.48642633746132 39.12168338286759),POINT (-75.4854779374699 39.12242969050662),POINT (-75.4844059307255 39.121565422859334),POINT (-75.48570711053159 39.12050631651153),...,0.025974,0.020779,0.028571,0.023377,0.020779,0.018182,0.015584,0.020779,0.015584,100010411001014
1,POINT (-75.49077914430639 39.12078128785075),POINT (-75.48959379103798 39.12150021599201),POINT (-75.49113486450344 39.122331145554234),POINT (-75.49024519555675 39.122917980464635),POINT (-75.48893419579453 39.12206613614668),POINT (-75.48918274224073 39.121798480298814),POINT (-75.4900007334724 39.1227944226747),POINT (-75.49153900111652 39.12208571960711),POINT (-75.49129276639798 39.12123920334929),POINT (-75.49105666027697 39.12114987737935),...,0.022167,0.019704,0.007389,0.024631,0.019704,0.009852,0.019704,0.022167,0.012315,100010411001007
2,POINT (-75.48216924981388 39.119858730056244),POINT (-75.48278997351336 39.11892758300041),POINT (-75.48343245254023 39.11959888490476),POINT (-75.48191643972886 39.12002455266947),POINT (-75.4816202143413 39.11989814423205),POINT (-75.48339487596786 39.11937374625381),POINT (-75.482594037113 39.11992193522498),POINT (-75.48234571426144 39.120047872546294),POINT (-75.48213109848535 39.12025070681873),POINT (-75.48231754205732 39.12017416969423),...,0.013055,0.036554,0.013055,0.023499,0.018277,0.023499,0.023499,0.010444,0.013055,100010411001018
3,POINT (-75.43650730519961 39.106332096195764),POINT (-75.44139189371703 39.1071066823113),POINT (-75.44056288263472 39.10316993224371),POINT (-75.43746085666307 39.10560277368679),POINT (-75.43950588435762 39.10684020953278),POINT (-75.43861290324428 39.10634446169378),POINT (-75.43956908556294 39.104260378858605),POINT (-75.44085826404074 39.10679553618104),POINT (-75.44101433668123 39.106844827257355),POINT (-75.44087968408004 39.10307038044667),...,0.021390,0.018717,0.021390,0.024064,0.029412,0.013369,0.021390,0.021390,0.010695,100010432021103
4,POINT (-75.52877120377036 39.16631467351781),POINT (-75.52891229905855 39.167076420679),POINT (-75.52884809510792 39.16703659303074),POINT (-75.52885066037625 39.16717935685959),POINT (-75.52825689817962 39.16642524495674),POINT (-75.5280092828465 39.16549663499765),POINT (-75.5287338250505 39.166418295774406),POINT (-75.52806821921656 39.16572078157774),POINT (-75.52829234271185 39.16602177257926),POINT (-75.5279662022453 39.16548946537896),...,0.016260,0.016260,0.013550,0.016260,0.018970,0.021680,0.013550,0.021680,0.018970,100010409001039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24110,POINT (-75.40180256544623 38.55001926636645),POINT (-75.4063194977567 38.54179576394802),POINT (-75.40073073179738 38.548380440595494),POINT (-75.40188737765594 38.54522871480124),POINT (-75.41247351626211 38.53868094462328),POINT (-75.40160167134562 38.53657791602348),POINT (-75.3877446438021 38.5361940289614),POINT (-75.40183538059539 38.54558841389241),POINT (-75.40236681518736 38.54827131388836),POINT (-75.4025189357522 38.53948036052502),...,0.021333,0.024000,0.018667,0.026667,0.016000,0.016000,0.021333,0.016000,0.021333,100050517011022
24111,POINT (-75.45453269459621 38.53843951210903),POINT (-75.45780873800224 38.53943742930966),POINT (-75.45455259439187 38.53969842328512),POINT (-75.46237391337114 38.539417371341486),POINT (-75.45233184311085 38.53825488533676),POINT (-75.46155868238117 38.53949318093205),POINT (-75.46646036070554 38.540040777025645),POINT (-75.45223128517115 38.53826713256842),POINT (-75.45757989577575 38.53543569015738),P

In [16]:
# Pull population data for 2010 Census blocks
# Define request parameters

year = '2010' # Year of interest
datasource = 'dec' # Survey name
subsource = 'pl' # Subsurvey name
GET = 'P001001,H001001,P001003' # Variables to query
FOR = 'block:*' # for predicate
IN = 'state:10&in=county:*&in=tract:*'

# Filepath to your Census API key
keyfile = 'CensusAPIKey.txt'

# Formatted API call
data_url = f'https://api.census.gov/data/{year}/{datasource}/{subsource}?get={GET}&for={FOR}&in={IN}'

# Read Census key into 'api_key'
with open(keyfile) as key:
    api_key = key.read().strip()

# Add key to url
data_url = f'{data_url}&key={api_key}'

# Request data and convert from json
data = requests.get(data_url).json()
# First entry in list is a list of variable names
data = pd.DataFrame(data[1:], columns = data[0])

# Rename columns to match shapefile pull
data.rename(columns = {"state":"STATEFP10", "county":"COUNTYFP10", "tract":"TRACTCE10", "block":"BLOCKCE10"}, inplace=True)

# Attach to block shapes
block_data = block_data.merge(data, on=["STATEFP10","COUNTYFP10","TRACTCE10","BLOCKCE10"])

In [17]:
# Pull population data for 2010 Census tracts
# Define request parameters

year = '2010' # Year of interest
datasource = 'dec' # Survey name
subsource = 'pl' # Subsurvey name
GET = 'P001001,H001001,P001003' # Variables to query
FOR = 'tract:*' # for predicate
IN = 'state:10' # in predicate


# Filepath to your Census API key
keyfile = 'CensusAPIKey.txt'

# Formatted API call
data_url = f'https://api.census.gov/data/{year}/{datasource}/{subsource}?get={GET}&for={FOR}&in={IN}'

# Read Census key into 'api_key'
with open(keyfile) as key:
    api_key = key.read().strip()

# Add key to url
data_url = f'{data_url}&key={api_key}'

# Request data and convert from json
data = requests.get(data_url).json()
# First entry in list is a list of variable names
data = pd.DataFrame(data[1:], columns = data[0])

# Rename columns to match shapefile pull
data.rename(columns = {"state":"STATE", "county":"COUNTY", "tract":"TRACT"}, inplace=True)

# Attach to tract shapes
tract_data = tract_data.merge(data, on=["STATE","COUNTY","TRACT"])

In [18]:
block_data

,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE10,GEOID10,NAME10,MTFCC10,UR10,UACE10,UATYP10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry,P001001,H001001,P001003
0,10,001,041100,1014,100010411001014,Block 1014,G5040,U,24580,U,S,50816,0,+39.1216358,-075.4858233,"POLYGON ((-75.48486 39.12239, -75.48481 39.122...",244,77,187
1,10,001,041100,1007,100010411001007,Block 1007,G5040,U,24580,U,S,48931,0,+39.1219647,-075.4904073,"POLYGON ((-75.49019 39.12340, -75.49005 39.123...",167,50,135
2,10,001,041100,1018,100010411001018,Block 1018,G5040,U,24580,U,S,28485,0,+39.1196396,-075.4826429,"POLYGON ((-75.48313 39.11849, -75.48333 39.118...",33,10,21
3,10,001,043202,1103,100010432021103,Block 1103,G5040,R,None,None,S,160376,0,+39.1054024,-075.4393957,"POLYGON ((-75.44219 39.10734, -75.43890 39.107...",14,6,12
4,10,001,040900,1039,100010409001039,Block 1039,G5040,U,24580,U,S,12254,0,+39.1662742,-075.5285219,"POLYGON ((-75.52849 39.16525, -75.52862 39.165...",53,27,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24110,10,005,051701,1022,100050517011022,Block 1022,G5040,R,None,None,S,2614704,0,+38.5418205,-075.4026864,"POLYGON ((-75.41186 38.54192, -75.41169 38.542...",1,1,1
24111,10,005,051701,1047,100050517011047,Block 1047,G5040,R,None,None,S,1331210,0,+38.5406585,-075.4582756,"POLYGON ((-75.45237 38.54473, -75.45222 38.544...",72,29,72
24112,10,005,051701,1049,100050517011049,Block 1049,G5040,R,None,None,S,130691,0,+38.5342417,-075.4535089,"POLYGON ((-75.45807 38.53492, -75.45771 38.535...",35,12,29
24113,10,005,051702,1120,100050517021120,Block 1120,G5040,R,None,None,S,155272,0,+38.5304973,-075.4586287,"POLYGON ((-75.45939 38.52753, -75.45941 38.527...",0,0,0


In [33]:
# Assign a fraction of the population of each block as a value to each point

# Merge each point to the 2010 census block containing it
population_per_point = point_cloud.merge(block_data, how="left", left_on="GEOID", right_on="GEOID10")

# Multiply each weight by the block population to get the block population per point
population_per_point[[x for x in population_per_point.columns if 'weight' in x]] = population_per_point[[x for x in population_per_point.columns if 'weight' in x]].mul(population_per_point["P001001"].astype(int), axis=0)

In [35]:
# Flatten to a GeoSeries where each row is a point and its weight
weights = np.array([[row["weight_" + str(i)] for i in range(n)] for _, row in population_per_point.iterrows()]).flatten()
points = np.array([[row["point_" + str(i)] for i in range(n)] for _, row in population_per_point.iterrows()]).flatten()
points_list = geopandas.GeoDataFrame({"population_per_point":weights,"geometry":points}, crs="EPSG:4269")


# Determine the number of points in the point cloud. This should be n * the number of census blocks
print(points_list.shape[0] / n == block_data.shape[0])

/Users/emiller/opt/anaconda3/envs/census-tract-redistricting/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/var/folders/9r/738fljg527g64ktptsnnzlpw0000gn/T/ipykernel_1191/679859926.py:3: FutureWarning: The input object of type 'Point' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Point', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  points = np.array([[row["point_" + str(i)] for i in range(n)] for _, row in population_per_point.iterrows()]).flatten()
/var/folders/9r/7

In [38]:
# Spatially join each point to the 2010 census tract containing it
variables_per_point = geopandas.sjoin(points_list, tract_data, how="left", op='within')

In [ ]:
# WARNING: Plot is large and should only be rendered if necessary
# TODO: Points around the edge of the state are being lost


# Find and plot all missed points 
missed_points = variables_per_point.loc[variables_per_point["index_right"].isna()]

# initialize the map and store it in a folium map object
us_map = folium.Map(location=[38.9108, -75.5277], zoom_start=8, tiles=None)

# Add background tiles
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(us_map)

# Style and highlight functions map population values to color values
style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':'red', 
                            'fillOpacity':0.75}

# Add a map over the tiles with the given colors and a tooltip
NIL=folium.features.GeoJson(
        missed_points, # Full geopandas data
        style_function=style_function, # function for base colors
        control=False
    )

# Add elements to map
us_map.add_child(NIL)

In [39]:
# Exclude missed points from the list
variables_per_point = variables_per_point.loc[~variables_per_point["index_right"].isna()]

# Divide variables of intersest by tract population and multiply by the portion of the population represented by each point
variables_per_point[["P001001", "H001001", "P001003"]] = variables_per_point[["P001001", "H001001", "P001003"]].astype(int).div(variables_per_point["P001001"].astype(int), axis=0).mul(variables_per_point["population_per_point"], axis=0)
# Reset index
variables_per_point = variables_per_point.reset_index()

In [40]:
variables_per_point

,index,population_per_point,geometry,index_right,GEO_ID,STATE,COUNTY,TRACT,NAME,LSAD,CENSUSAREA,P001001,H001001,P001003
0,0,4.436364,POINT (-75.48551 39.12224),87.0,1400000US10001041100,10,001,041100,411,Tract,6.190,4.436364,1.360564,3.218143
1,1,6.337662,POINT (-75.48628 39.12054),87.0,1400000US10001041100,10,001,041100,411,Tract,6.190,6.337662,1.943663,4.597347
2,2,5.703896,POINT (-75.48669 39.12179),87.0,1400000US10001041100,10,001,041100,411,Tract,6.190,5.703896,1.749296,4.137612
3,3,4.436364,POINT (-75.48616 39.12191),87.0,1400000US10001041100,10,001,041100,411,Tract,6.190,4.436364,1.360564,3.218143
4,4,3.802597,POINT (-75.48521 39.12252),87.0,1400000US10001041100,10,001,041100,411,Tract,6.190,3.802597,1.166198,2.758408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1198150,1205745,0.000000,POINT (-75.45989 38.52478),176.0,1400000US10005051702,10,005,051702,517.02,Tract,68.279,0.000000,0.000000,0.000000
1198151,1205746,0.000000,POINT (-75.46369 38.53136),176.0,1400000US10005051702,10,005,051702,517.02,Tract,68.279,0.000000,0.000000,0.000000
1198152,1205747,0.000000,POINT (-75.47245 38.52963),176.0,1400000US10005051702,10,005,051702,517.02,Tract,68.279,0.000000,0.000000,0.000000
1198153,1205748,0.000000,POINT (-75.46253 38.52081),176.0,1400000US10005051702,10,005,051702,517.02,Tract,68.279,0.000000,0.000000,0.000000


In [41]:
# Print the number of points missed in the transfer of data from tracts to points
print(points_list.shape[0] -  variables_per_point.shape[0])

7595


In [42]:
# Request shapefile data for 2020 census tracts and convert to geopandas dataframe

# Shapefile url
data_url = 'https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_10_tract_500k.zip'


# Request data
data = requests.get(data_url)
# convert to pandas dataframe
tract2020 = geopandas.read_file(BytesIO(data.content))

In [43]:
# Spatially join points to 2020 census tracts
interpolated_values = geopandas.sjoin(variables_per_point[["GEO_ID","geometry","P001001","H001001","P001003"]], tract2020, how="left", op='within')

# Sum the values for each 2020 tract
interpolated_values = interpolated_values[["GEOID", "P001001", "H001001", "P001003"]].groupby("GEOID").sum()

interpolated_values


#tract2020

,P001001,H001001,P001003
GEOID,,,
10001040100,6544.629384,2470.397083,5705.400048
10001040201,5043.341307,2023.650121,3666.474518
10001040203,5005.945996,2017.406765,3116.138075
10001040204,4655.971419,1733.185423,3160.993697
10001040205,2878.966888,1071.802060,1954.369020
...,...,...,...
10005051702,5618.467267,2305.259650,5017.160632
10005051801,4900.095310,2123.542025,3863.847523
10005051802,4163.679323,1744.174047,2486.972903


In [44]:
# Pull population data for 2020 Census tracts
# Define request parameters

year = '2020' # Year of interest
datasource = 'dec' # Survey name
subsource = 'pl' # Subsurvey name
GET = 'P1_001N,H1_001N,P1_003N' # Variables to query
FOR = 'tract:*' # for predicate
IN = 'state:10' # in predicate


# Filepath to your Census API key
keyfile = 'CensusAPIKey.txt'

# Formatted API call
data_url = f'https://api.census.gov/data/{year}/{datasource}/{subsource}?get={GET}&for={FOR}&in={IN}'

# Read Census key into 'api_key'
with open(keyfile) as key:
    api_key = key.read().strip()

# Add key to url
data_url = f'{data_url}&key={api_key}'

# Request data and convert from json
data = requests.get(data_url).json()
# First entry in list is a list of variable names
tract2020_data = pd.DataFrame(data[1:], columns = data[0])

# Add a GEOID column to the data
tract2020_data["GEOID"] = tract2020_data["state"].astype(str) + tract2020_data["county"].astype(str) +tract2020_data["tract"].astype(str)

In [45]:
# Write combined dataframe of 2020 ground truth and estimated values to a csv
interpolated_values.merge(tract2020_data, left_index=True, right_on="GEOID").to_csv("estimates.csv", index=False)

In [46]:
interpolated_values.merge(tract2020_data, left_index=True, right_on="GEOID")

,P001001,H001001,P001003,P1_001N,H1_001N,P1_003N,state,county,tract,GEOID
220,6544.629384,2470.397083,5705.400048,7315,2740,5980,10,001,040100,10001040100
221,5043.341307,2023.650121,3666.474518,5446,2123,3424,10,001,040201,10001040201
222,5005.945996,2017.406765,3116.138075,5182,2157,2808,10,001,040203,10001040203
223,4655.971419,1733.185423,3160.993697,6451,2269,3613,10,001,040204,10001040204
224,2878.966888,1071.802060,1954.369020,4699,1985,2430,10,001,040205,10001040205
...,...,...,...,...,...,...,...,...,...,...
214,5618.467267,2305.259650,5017.160632,6577,2590,5286,10,005,051702,10005051702
215,4900.095310,2123.542025,3863.847523,5359,2154,3636,10,005,051801,10005051801
216,4163.679323,1744.174047,2486.972903,4354,1740,2256,10,005,051802,10005051802
217,4553.284160,1828.295699,3622.678657,4760,1949,3566,10,005,051900,10005051900
